# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
!pip install gmaps


In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [27]:
city_df= pd.read_csv('../output_data/cities.csv')
city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city_id     44 non-null     int64  
 1   city        44 non-null     object 
 2   lat         44 non-null     float64
 3   maxtemp     44 non-null     float64
 4   lng         44 non-null     float64
 5   humidity    44 non-null     int64  
 6   cloudiness  44 non-null     int64  
 7   windspeed   44 non-null     float64
 8   country     44 non-null     object 
 9   date        44 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 3.6+ KB


In [22]:
gmaps.configure(api_key= g_key)
locations= city_df[['lat', 'lng']]
# locations

In [24]:
humidity= city_df['humidity']
# humidity

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key= g_key)

fig= gmaps.figure()
heatmap= gmaps.heatmap_layer(locations, weights= humidity, dissipating= False, max_intensity= 300, point_radius= 4)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
beautycity_df= city_df.loc[(city_df['maxtemp']<90)&(city_df['maxtemp']>20)&(city_df['windspeed']<5)&(city_df['cloudiness']!=0)].dropna()
beautycity_df

,city_id,city,lat,maxtemp,lng,humidity,cloudiness,windspeed,country,date
1,1,vaini,-21.2000,26.09,-175.2000,89,20,2.06,TO,1644769727
3,3,bria,6.5423,35.44,21.9863,12,95,3.60,CF,1644769730
4,4,augusto correa,-1.0217,28.08,-46.6350,77,48,2.91,BR,1644769731
7,7,rodrigues alves,-7.7380,25.11,-72.6509,81,100,1.02,BR,1644769735
15,15,saint-philippe,-21.3585,26.77,55.7679,86,53,3.91,RE,1644769745
18,18,castro,-24.7911,26.58,-50.0119,53,7,2.80,BR,1644769749
21,21,cabo san lucas,22.8909,25.34,-109.9124,44,100,1.54,MX,1644769529
23,23,rikitea,-23.1203,25.86,-134.9692,70,8,4.87,PF,1644769653
24,24,valdivia,-39.8142,24.05,-73.2459,32,96,1.98,CL,1644769756
27,27,george town,5.4112,27.96,100.3354,91,40,3.09,MY,1644769579


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df= beautycity_df[['city', 'country', 'lat', 'lng']].copy()
hotel_df["Hotel Name"]= ''
hotel_df

/var/folders/b8/5ywcdwz526nc7m7jzyf5rwnm0000gn/T/ipykernel_4474/3721380660.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]= ''


,city,country,lat,lng,Hotel Name
1,vaini,TO,-21.2000,-175.2000,
3,bria,CF,6.5423,21.9863,
4,augusto correa,BR,-1.0217,-46.6350,
7,rodrigues alves,BR,-7.7380,-72.6509,
15,saint-philippe,RE,-21.3585,55.7679,
18,castro,BR,-24.7911,-50.0119,
21,cabo san lucas,MX,22.8909,-109.9124,
23,rikitea,PF,-23.1203,-134.9692,
24,valdivia,CL,-39.8142,-73.2459,
27,george town,MY,5.4112,100.3354,


In [12]:

# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": 'lodging',
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat= row['lat']
    lng= row['lng']
    params['location']= f'{lat}, {lng}'
    
# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    address= response.json()
    try:
        hotel_df.loc[index, 'hotelname']= address['results'][0]['name']
    except:
         print("data not found")
hotel_df

/Users/adityabagchi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/adityabagchi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,city,country,lat,lng,Hotel Name,hotelname
1,vaini,TO,-21.2000,-175.2000,,Keleti Beach Resort
3,bria,CF,6.5423,21.9863,,WFP Guesthouse
4,augusto correa,BR,-1.0217,-46.6350,,Fazenda Bacuri
7,rodrigues alves,BR,-7.7380,-72.6509,,Pousada Aconchego
15,saint-philippe,RE,-21.3585,55.7679,,"Chambres d'hôte ""La Trinité"""
18,castro,BR,-24.7911,-50.0119,,CHACARA BAILLY
21,cabo san lucas,MX,22.8909,-109.9124,,Hotel Tesoro Los Cabos
23,rikitea,PF,-23.1203,-134.9692,,People ThankYou
24,valdivia,CL,-39.8142,-73.2459,,Hostal Entre Rios
27,george town,MY,5.4112,100.3354,,Cititel Penang


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [29]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))